In [ ]:
!pip install torch torchvision transformers
#load pretrained bert base model
from transformers import BertModel

bert_model = BertModel.from_pretrained('bert-base-uncased')

print("Done loading BERT model.")

     |████████████████████████████████| 2.1MB 25.3MB/s 
     |████████████████████████████████| 901kB 49.5MB/s 
     |████████████████████████████████| 3.3MB 40.3MB/s 



Done loading BERT model.


In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [ ]:
import json
import pandas as pd
import numpy as np
import re
import nltk
nltk.download('stopwords')
from nltk import collections
from nltk.tag import pos_tag
from nltk.corpus import wordnet
from nltk.tokenize import TweetTokenizer
from nltk.corpus import stopwords
from nltk.tokenize import RegexpTokenizer

from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.naive_bayes import MultinomialNB
from sklearn.linear_model import LogisticRegression
from sklearn import svm, metrics
from sklearn.metrics import precision_recall_fscore_support
from sklearn.svm import SVC
from nltk.stem import LancasterStemmer, WordNetLemmatizer


[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Unzipping corpora/stopwords.zip.


In [ ]:
def process_data(read_data_filepath, read_label_filepath):
  label_map = {'non-rumour': 0, 'rumour': 1}
  index_map = {v: k for k, v in label_map.items()}
  with open(read_data_filepath, 'r') as f:
    source_id = []
    text = []
    label = [] 
    for line in f:
      line = json.loads(line)
      source_id.append(line[0]['id_str'])
      text.append(line[0]['text'])
    df_data = pd.DataFrame({'id':source_id,'text':text, 'label':None})
  if read_label_filepath is not None:
    with open(read_label_filepath, 'r') as f:
      id, label = [], []
      temp = json.loads(f.read())
      for key, val in temp.items():
        id.append(key)
        label.append(val)
      # label_data = pd.DataFrame(list(zip(id, label)))
      # label_data.columns = ['id', 'label']
      for i in range(len(label)):
          label[i] = 1 if label[i] == "rumour" else 0 #convert_label(label_data["label"][i])
          if source_id[i] == id[i]:
            df_data.label[i] = label[i]
    return df_data
  else:
    return df_data[['id', 'text']]

df_dev = process_data('/content/drive/My Drive/data/dev.data.jsonl', '/content/drive/My Drive/data/dev.label.json')
df_train = process_data('/content/drive/My Drive/data/train.data.jsonl', '/content/drive/My Drive/data/train.label.json')
df_test = process_data('/content/drive/My Drive/data/test.data.jsonl', None)
# df_covid = process_data('/content/drive/My Drive/data/covid.data.jsonl', None)

# a list of events, and each event is a list of tweets (source tweet + reactions)
print("Number of train data =", len(df_train["text"]))
print("Number of dev data =", len(df_dev["text"]))
print("Number of test data =", len(df_test["text"]))
# print("Number of test data =", len(df_covid["text"]))

# print(type(df_test['id'][0]))


Number of train data = 4641
Number of dev data = 580
Number of test data = 581


In [ ]:
nltk.download('stopwords')
nltk.download('wordnet')

# clean data
default_stopwords = set(nltk.corpus.stopwords.words('english'))
# will have to add the following custom
custom_stopwords = {"http://", "rt", "co", "https://", "www", "@"}
all_stopwords = default_stopwords | custom_stopwords
eng_stemmer = nltk.stem.SnowballStemmer('english')
tt = TweetTokenizer()
lemmatizer = WordNetLemmatizer()


def preprocess_data(df):
    print("Started preprocessing!")
    text = df['text'].apply(str)
    for i in range(len(text)):
        text[i] = re.sub(r'((www\.[^\s]+)|(https?://[^\s]+))', '', text[i])
        text[i] = re.sub(r'@[^\s]+', '', text[i])
        text[i] = tt.tokenize(text[i])
        # remove single character words
        text[i] = [word for word in text[i] if len(word) > 1]
        # convert to lower case
        text[i] = [word.lower() for word in text[i]]
        # removing numbers
        text[i] = [word for word in text[i] if word.isalpha()]
        # stem the words
        text[i] = [lemmatizer.lemmatize(word) for word in text[i]]
        # remove stopwords
        text[i] = [word for word in text[i] if word not in default_stopwords]
        text[i] = " ".join(text[i])
        df['text'][i] = text[i]
    return df['text']

[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!
[nltk_data] Downloading package wordnet to /root/nltk_data...
[nltk_data]   Unzipping corpora/wordnet.zip.


In [ ]:
# # clean data
# nltk.download('stopwords')
# default_stopwords = set(nltk.corpus.stopwords.words('english'))
# # will have to add the following custom
# # custom_stopwords = {"http://", "rt", "co", "https://", "www", "@"}
# all_stopwords = default_stopwords # | custom_stopwords
# eng_stemmer = nltk.stem.SnowballStemmer('english')
# tt = TweetTokenizer() # strip_handles=True, reduce_len=True
# lemmatizer = WordNetLemmatizer()

# contraction_patterns = [(r'won\'t', 'will not'), (r'can\'t', 'cannot'), (r'i\'m', 'i am'), (r'ain\'t', 'is not'),
#                         (r'(\w+)\'ll', '\g<1> will'), (r'(\w+)n\'t', '\g<1> not'),
#                         (r'(\w+)\'ve', '\g<1> have'), (r'(\w+)\'s', '\g<1> is'), (r'(\w+)\'re', '\g<1> are'),
#                         (r'(\w+)\'d', '\g<1> would'), (r'&', 'and'), (r'dammit', 'damn it'), (r'dont', 'do not'),
#                         (r'wont', 'will not')]
# def replaceContraction(text):
#     patterns = [(re.compile(regex), repl) for (regex, repl) in contraction_patterns]
#     for (pattern, repl) in patterns:
#         (text, count) = re.subn(pattern, repl, text)
#     return text

# def preprocess_data(df):
#     print("Started preprocessing!")
#     text = df['text']
#     for i in range(len(text)):
#         # text[i] = re.sub(r"[^A-Za-z0-9(),!?#@\'\`]", " ", text[i])
#         # replace AtUser
#         text[i] = re.sub(r'@[^\s]+', '', text[i])
#         # replace url
#         text[i] = re.sub(r'((www\.[^\s]+)|(https?://[^\s]+))', '', text[i])
#         # remove Hashtag In Front Of Word
#         text[i] = re.sub(r'#([^\s]+)', r'\1', text[i])
#         # remove ? ! .
#         text[i] = re.sub(r"!", " ! ", text[i])
#         text[i] = re.sub(r"\?", " ? ", text[i])
#         text[i] = re.sub(r"\.", ' ', text[i])
#         # remove double space
#         text[i] = re.sub(r'\s+', ' ', text[i]).strip()
#         # Replaces contractions from a string to their equivalents
#         text[i] = replaceContraction(text[i])
#         # Remove puncuation
#         # text[i] = text[i].translate(string.punctuation)
#         text[i] = tt.tokenize(text[i])
#         ## Stemming
#         # remove single character words
#         text[i] = [word for word in text[i] if len(word) > 1]
#         # removing numbers
#         text[i] = [word for word in text[i] if not word.isnumeric()]
#         # convert to lower case
#         text[i] = [word.lower() for word in text[i]]
#         # stem the words
#         # text[i] = [lemmatizer.lemmatize(word) for word in text[i]]
#         text[i] = [eng_stemmer.stem(word) for word in text[i]]

#         # # remove stopwords
#         text[i] = [word for word in text[i] if word not in all_stopwords]
#         # text[i] = [word for word in text[i] if word not in string.punctuation]

#         text[i] = " ".join(text[i])
#         df['text'][i] = text[i]
#     return df['text']


train_texts = preprocess_data(df_train)
dev_texts = preprocess_data(df_dev)
test_texts = preprocess_data(df_test)
# covid_texts = preprocess_data(df_covid)

df_train_to_csv = pd.DataFrame({'id': df_train["id"].apply(str), 'sentence': train_texts, 'label': df_train["label"]})
df_train_to_csv.to_csv('train.tsv', sep='\t', index=False)

df_dev_to_csv = pd.DataFrame({'id': df_dev["id"].apply(str), 'sentence': dev_texts, 'label': df_dev["label"]})
df_dev_to_csv.to_csv('dev.tsv', sep='\t', index=False)

df_test_to_csv = pd.DataFrame({'id': df_test["id"].apply(str), 'sentence': test_texts})
df_test_to_csv.to_csv('test.tsv', sep='\t', index=False)

# df_covid_to_csv = pd.DataFrame({'id': df_covid["id"].apply(str), 'sentence': covid_texts})
# df_covid_to_csv.to_csv('covid.tsv', sep='\t', index=False)

## Get Train data and Validation data for model 1
# x_train, x_test, y_train, y_test = train_test_split(train_texts, np.array(df_train["label"], dtype=int), test_size=0.1)
x_train = train_texts
y_train = np.array(df_train["label"])

x_dev = dev_texts
y_dev = np.array(df_dev["label"])

x_test = test_texts

# x_covid = covid_texts

Started preprocessing!
Started preprocessing!
Started preprocessing!


In [ ]:
# print(dev_texts[579])
# print(x_dev[579])
# print(df_test.text[580])

In [ ]:
!pip install torch torchvision transformers
import torch
from torch.utils.data import Dataset
from transformers import BertTokenizer
import pandas as pd

class SSTDataset(Dataset):

    def __init__(self, filename, maxlen):

        #Store the contents of the file in a pandas dataframe
        self.df = pd.read_csv(filename, delimiter = '\t').fillna("")
        # print(self.df['sentence'].tail(10))
        #Initialize the BERT tokenizer
        self.tokenizer = BertTokenizer.from_pretrained('bert-base-uncased')

        self.maxlen = maxlen

    def __len__(self):
        return len(self.df)

    def __getitem__(self, index):

        #Selecting the sentence and label at the specified index in the data frame

        label = self.df.loc[index, 'label']
        sentence = self.df.loc[index, 'sentence']

        #Preprocessing the text to be suitable for BERT
        tokens = self.tokenizer.tokenize(sentence) #Tokenize the sentence
        tokens = ['[CLS]'] + tokens + ['[SEP]'] #Insering the CLS and SEP token in the beginning and end of the sentence
        if len(tokens) < self.maxlen:
            tokens = tokens + ['[PAD]' for _ in range(self.maxlen - len(tokens))] #Padding sentences
        else:
            tokens = tokens[:self.maxlen-1] + ['[SEP]'] #Prunning the list to be of specified max length

        tokens_ids = self.tokenizer.convert_tokens_to_ids(tokens) #Obtaining the indices of the tokens in the BERT Vocabulary
        tokens_ids_tensor = torch.tensor(tokens_ids) #Converting the list to a pytorch tensor

        #Obtaining the attention mask i.e a tensor containing 1s for no padded tokens and 0s for padded ones
        attn_mask = (tokens_ids_tensor != 0).long()

        return tokens_ids_tensor, attn_mask, label


class SSTDataset1(Dataset):

    def __init__(self, filename, maxlen):

        #Store the contents of the file in a pandas dataframe
        self.df = pd.read_csv(filename, delimiter = '\t').fillna("")
        print(self.df['sentence'].tail(10))
        self.label_map = {'non-rumour': 0, 'rumour': 1}
        #Initialize the BERT tokenizer
        self.tokenizer = BertTokenizer.from_pretrained('bert-base-uncased')

        self.maxlen = maxlen

    def __len__(self):
        return len(self.df)

    def __getitem__(self, index):

        #Selecting the sentence and label at the specified index in the data frame
        # id = self.df.loc[index, 'id']
        sentence = self.df.loc[index, 'sentence']

        #Preprocessing the text to be suitable for BERT
        tokens = self.tokenizer.tokenize(sentence) #Tokenize the sentence
        tokens = ['[CLS]'] + tokens + ['[SEP]'] #Insering the CLS and SEP token in the beginning and end of the sentence
        if len(tokens) < self.maxlen:
            tokens = tokens + ['[PAD]' for _ in range(self.maxlen - len(tokens))] #Padding sentences
        else:
            tokens = tokens[:self.maxlen-1] + ['[SEP]'] #Prunning the list to be of specified max length

        tokens_ids = self.tokenizer.convert_tokens_to_ids(tokens) #Obtaining the indices of the tokens in the BERT Vocabulary
        tokens_ids_tensor = torch.tensor(tokens_ids) #Converting the list to a pytorch tensor

        #Obtaining the attention mask i.e a tensor containing 1s for no padded tokens and 0s for padded ones
        attn_mask = (tokens_ids_tensor != 0).long()
        return tokens_ids_tensor, attn_mask


In [ ]:
from torch.utils.data import DataLoader

#Creating instances of training and development set
#maxlen sets the maximum length a sentence can have
#any sentence longer than this length is truncated to the maxlen size
train_set = SSTDataset(filename = 'train.tsv', maxlen = 100)
dev_set = SSTDataset(filename = 'dev.tsv', maxlen = 100)
test_set = SSTDataset1(filename = 'test.tsv', maxlen = 100)
# covid_set = SSTDataset1(filename = 'covid.tsv', maxlen = 100)
#Creating intsances of training and development dataloaders
train_loader = DataLoader(train_set, batch_size = 64, num_workers = 2)
dev_loader = DataLoader(dev_set, batch_size = 64, num_workers = 2)
test_loader = DataLoader(test_set, batch_size = 600, num_workers = 2)
# covid_loader = DataLoader(covid_set, batch_size = 1000, num_workers = 2)
print("Done preprocessing training and development data.")


571    mourner hold sign showing victim eye rally ny ...
572    sorry tweeting cut defy islamism pretend coura...
573    breaking least two reportedly killed hostage d...
574            trended worldwide much jewish blood cheap
575    city hamilton offer condolence family cpl nath...
576    hearing gunfire siege printing building dammar...
577    feel though killing someone pen stephane charb...
578    must confirm deepest regret germanwings flight...
579    protestors blocked west florissant major stree...
580    terrible news ottawa today thought prayer ever...
Name: sentence, dtype: object
Done preprocessing training and development data.


In [ ]:
import torch
import torch.nn as nn
from transformers import BertModel

class SentimentClassifier(nn.Module):

    def __init__(self):
        super(SentimentClassifier, self).__init__()
        #Instantiating BERT model object 
        self.bert_layer = BertModel.from_pretrained('bert-base-uncased')
        
        #Classification layer
        #input dimension is 768 because [CLS] embedding has a dimension of 768
        #output dimension is 1 because we're working with a binary classification problem
        self.cls_layer = nn.Linear(768, 1)

    def forward(self, seq, attn_masks):
        '''
        Inputs:
            -seq : Tensor of shape [B, T] containing token ids of sequences
            -attn_masks : Tensor of shape [B, T] containing attention masks to be used to avoid contibution of PAD tokens
        '''

        #Feeding the input to BERT model to obtain contextualized representations
        outputs = self.bert_layer(seq, attention_mask = attn_masks)
        cont_reps = outputs.last_hidden_state

        #Obtaining the representation of [CLS] head (the first token)
        cls_rep = cont_reps[:, 0]

        #Feeding cls_rep to the classifier layer
        logits = self.cls_layer(cls_rep)

        return logits

In [ ]:
gpu = 0 #gpu ID

print("Creating the sentiment classifier, initialised with pretrained BERT-BASE parameters...")
net = SentimentClassifier()
net.cuda(gpu) #Enable gpu support for the model
print("Done creating the sentiment classifier.")

Creating the sentiment classifier, initialised with pretrained BERT-BASE parameters...
Done creating the sentiment classifier.


In [ ]:
import torch.nn as nn
import torch.optim as optim

criterion = nn.BCEWithLogitsLoss()
opti = optim.Adam(net.parameters(), lr = 2e-5)

In [ ]:
import time

def train(net, criterion, opti, train_loader, dev_loader, max_eps, gpu):

    best_acc = 0
    best_f1 = 0
    st = time.time()
    for ep in range(max_eps):
        
        for it, (seq, attn_masks, labels) in enumerate(train_loader):
            #Clear gradients
            opti.zero_grad()  
            #Converting these to cuda tensors
            seq, attn_masks, labels = seq.cuda(gpu), attn_masks.cuda(gpu), labels.cuda(gpu)

            #Obtaining the logits from the model
            logits = net(seq, attn_masks)

            #Computing loss
            loss = criterion(logits.squeeze(-1), labels.float())

            #Backpropagating the gradients
            loss.backward()

            #Optimization step
            opti.step()
              
            if it % 100 == 0:
                
                acc = get_accuracy_from_logits(logits, labels)
                print("Iteration {} of epoch {} complete. Loss: {}; Accuracy: {}; Time taken (s): {}".format(it, ep, loss.item(), acc, (time.time()-st)))                
                st = time.time()
        print("this is test length: ", len(predict(net, test_loader, gpu)))
        # print("this is covid length: ", len(predict(net, covid_loader, gpu)))
        dev_acc, dev_loss, dev_f1 = evaluate(net, criterion, dev_loader, gpu)
        """
        test
        """
        y_test_pred = predict(net, test_loader, gpu).squeeze()
        index_map = {v: k for k, v in test_set.label_map.items()}
        df = pd.DataFrame({"label": y_test_pred.tolist()})
        df['label'] = df.label.apply(lambda x: index_map[x])
        df_pred = pd.concat([test_set.df.loc[:, ['id']], df.loc[:, 'label']], axis=1)
        print(ep)
        display(df_pred)
        filename = 'test-' + str(ep) + '.json'
        write_pred(df_pred["id"].apply(str), df_pred['label'], filename)
        """
        covid
        """
        # y_covid_pred = predict(net, covid_loader, gpu).squeeze()
        # index_map = {v: k for k, v in covid_set.label_map.items()}
        # dfcovid = pd.DataFrame({"label": y_covid_pred.tolist()})
        # dfcovid['label'] = df.label.apply(lambda x: index_map[x])
        # dfcovid_pred = pd.concat([test_set.dfcovid.loc[:, ['id']], dfcovid.loc[:, 'label']], axis=1)
        # print(ep)
        # display(dfcovid_pred)
        # filename = 'test-' + str(ep) + '.json'
        # write_pred(dfcovid_pred["id"].apply(str), dfcovid_pred['label'], filename)

        print("Epoch {} complete! Development f1 score: {}; Development Loss: {}".format(ep, dev_f1, dev_loss))
        if dev_f1 > best_f1:
            print("Best development f1 score improved from {} to {}, saving model...".format(best_f1, dev_f1))
            best_f1 = dev_f1
            torch.save(net.state_dict(), 'sstcls_{}.dat'.format(ep))


In [ ]:
import numpy as np

def f1_loss(y_true:torch.Tensor, y_pred:torch.Tensor, is_training=False) -> torch.Tensor:
    '''Calculate F1 score. Can work with gpu tensors
    
    The original implmentation is written by Michal Haltuf on Kaggle.
    
    Returns
    -------
    torch.Tensor
        `ndim` == 1. 0 <= val <= 1
    
    Reference
    ---------
    - https://www.kaggle.com/rejpalcz/best-loss-function-for-f1-score-metric
    - https://scikit-learn.org/stable/modules/generated/sklearn.metrics.f1_score.html#sklearn.metrics.f1_score
    - https://discuss.pytorch.org/t/calculating-precision-recall-and-f1-score-in-case-of-multi-label-classification/28265/6
    
    '''
    assert y_true.ndim == 1
    assert y_pred.ndim == 1 or y_pred.ndim == 2
    
    if y_pred.ndim == 2:
        y_pred = y_pred.argmax(dim=1)
        
    tp = (y_true * y_pred).sum().to(torch.float32)
    tn = ((1 - y_true) * (1 - y_pred)).sum().to(torch.float32)
    fp = ((1 - y_true) * y_pred).sum().to(torch.float32)
    fn = (y_true * (1 - y_pred)).sum().to(torch.float32)
    
    epsilon = 1e-7
    
    precision = tp / (tp + fp + epsilon)
    recall = tp / (tp + fn + epsilon)
    
    f1 = 2* (precision*recall) / (precision + recall + epsilon)
    f1.requires_grad = is_training
    return f1


def get_f1_score_from_logits(logits, labels):
    probs = torch.sigmoid(logits.unsqueeze(-1))
    soft_probs = (probs > 0.5).long()
    f1_score = f1_loss(labels, soft_probs.squeeze())
    return f1_score


def get_accuracy_from_logits(logits, labels):
    probs = torch.sigmoid(logits.unsqueeze(-1))
    soft_probs = (probs > 0.5).long()
    acc = (soft_probs.squeeze() == labels).float().mean()    
    return acc

def evaluate(net, criterion, dataloader, gpu):
    net.eval()

    mean_acc, mean_loss, mean_f1 = 0, 0, 0
    count = 0

    with torch.no_grad():
        for seq, attn_masks, labels in dataloader:
            seq, attn_masks, labels = seq.cuda(gpu), attn_masks.cuda(gpu), labels.cuda(gpu)
            logits = net(seq, attn_masks)
            mean_loss += criterion(logits.squeeze(-1), labels.float()).item()
            mean_acc += get_accuracy_from_logits(logits, labels)
            mean_f1 += get_f1_score_from_logits(logits, labels)
            count += 1

    return mean_acc / count, mean_loss / count, mean_f1 / count


def predict(net, test_loader, gpu):
    net.eval()
    # pred_label=[]
    predictions = None
    with torch.no_grad():

        for seq, attn_masks in test_loader:
            seq, attn_masks = seq.cuda(gpu), attn_masks.cuda(gpu)
            logits = net(seq, attn_masks)
            probs = torch.sigmoid(logits.unsqueeze(-1))
            soft_probs = (probs > 0.5).long()
            # pred_label.append(soft_probs)
            # soft_probs = soft_probs.squeeze()
            if predictions is None:
                predictions = soft_probs.data
            else:
                predictions = torch.cat((predictions, soft_probs.data), 1)
    return predictions.data 


def write_pred(id, labels, filename):
    dic = collections.OrderedDict()
    for i in range(len(id)):
        dic[id[i]] = labels[i]
    with open(filename, 'w') as f:
        json.dump(dic, f)
    print("save finished")

In [ ]:
num_epoch = 10
#fine-tune the model
train(net, criterion, opti, train_loader, dev_loader, num_epoch, gpu)

Iteration 0 of epoch 0 complete. Loss: 0.750972330570221; Accuracy: 0.375; Time taken (s): 1.2791929244995117
this is test length:  581
0


,id,label
0,544382249178001408,rumour
1,525027317551079424,rumour
2,544273220128739329,rumour
3,499571799764770816,non-rumour
4,552844104418091008,non-rumour
...,...,...
576,553581227165642752,non-rumour
577,552816302780579840,non-rumour
578,580350000074457088,rumour
579,498584409055174656,non-rumour


save finished
Epoch 0 complete! Development f1 score: 0.7018754482269287; Development Loss: 0.3403978407382965
Best development f1 score improved from 0 to 0.7018754482269287, saving model...
Iteration 0 of epoch 1 complete. Loss: 0.3626158535480499; Accuracy: 0.84375; Time taken (s): 82.3402738571167
this is test length:  581
1


,id,label
0,544382249178001408,rumour
1,525027317551079424,rumour
2,544273220128739329,rumour
3,499571799764770816,non-rumour
4,552844104418091008,non-rumour
...,...,...
576,553581227165642752,non-rumour
577,552816302780579840,non-rumour
578,580350000074457088,rumour
579,498584409055174656,non-rumour


save finished
Epoch 1 complete! Development f1 score: 0.6851293444633484; Development Loss: 0.3847378045320511
Iteration 0 of epoch 2 complete. Loss: 0.23585842549800873; Accuracy: 0.875; Time taken (s): 81.78643870353699
this is test length:  581
2


...

save finished
Epoch 7 complete! Development f1 score: 0.7249711155891418; Development Loss: 0.7265150964260101
Best development f1 score improved from 0.7246204614639282 to 0.7249711155891418, saving model...
Iteration 0 of epoch 8 complete. Loss: 0.004165038000792265; Accuracy: 1.0; Time taken (s): 83.18457627296448
this is test length:  581
8


,id,label
0,544382249178001408,rumour
1,525027317551079424,rumour
2,544273220128739329,rumour
3,499571799764770816,non-rumour
4,552844104418091008,non-rumour
...,...,...
576,553581227165642752,non-rumour
577,552816302780579840,non-rumour
578,580350000074457088,rumour
579,498584409055174656,non-rumour


save finished
Epoch 8 complete! Development f1 score: 0.7121772766113281; Development Loss: 0.7441752851009369
Iteration 0 of epoch 9 complete. Loss: 0.0013350308872759342; Accuracy: 1.0; Time taken (s): 82.04450726509094
this is test length:  581
9


,id,label
0,544382249178001408,rumour
1,525027317551079424,rumour
2,544273220128739329,rumour
3,499571799764770816,non-rumour
4,552844104418091008,non-rumour
...,...,...
576,553581227165642752,non-rumour
577,552816302780579840,non-rumour
578,580350000074457088,rumour
579,498584409055174656,non-rumour


save finished
Epoch 9 complete! Development f1 score: 0.7050656080245972; Development Loss: 0.7968592703342438


In [ ]:
# y_test_pred = predict(net, test_loader, gpu).squeeze()

In [ ]:
# index_map = {v: k for k, v in test_set.label_map.items()}
# df = pd.DataFrame({"label": y_test_pred.tolist()})
# df['label'] = df.label.apply(lambda x: index_map[x])
# df_pred = pd.concat([test_set.df.loc[:, ['id']], df.loc[:, 'label']], axis=1)
# write_pred(df_pred["id"].apply(str), df_pred['label'], "test-output.json")


